In [32]:
import cv2, requests
import pandas as pd
import urllib.request
import numpy as np
import os
import csv
# import face_recognition

from face_crop_plus import Cropper
from deepface import DeepFace
from torch.cuda import is_available

In [29]:
df_person = pd.read_csv('./newdb/tagPerson.csv')
tagged_person_list = df_person.name[0], df_person.name[50], df_person.name[88]
tagged_person_id_list = [0, 50, 88]
# Chia-wei Woo (吳家瑋), Nancy IP (葉玉如), Paul CHU (朱經武)


In [47]:
df_list = []
df_tag = pd.read_csv('./newdb/tag.csv')

result_df = df_tag[df_tag['personID'] == 0].dropna() # change: 
result_df.reset_index(inplace=True)
    
result_df


,index,tagID,albumID,photoID,personID,c1,c2,c3,c4
0,179,179,685.0,366-1,0.0,504.0,112.0,48.0,55.0
1,197,197,271.0,CONF HERA_004_SM,0.0,306.0,224.0,71.0,87.0
2,212,212,366.0,CER KEL_003_SM,0.0,314.0,76.0,96.0,116.0
3,214,214,369.0,CER W3C3_001_SM,0.0,272.0,33.0,80.0,108.0
4,230,230,393.0,CONG 7_044,0.0,109.0,19.0,228.0,210.0
5,242,242,449.0,EXH MIA01_X4_SM,0.0,147.0,89.0,75.0,66.0
6,263,263,481.0,DON SOC_001_SC,0.0,382.0,126.0,82.0,102.0
7,287,287,564.0,DIS AL_001_SM,0.0,334.0,19.0,130.0,128.0
8,288,288,564.0,DIS AL_002_SM,0.0,314.0,16.0,141.0,130.0
9,300,300,585.0,DIS HKE__036_SM,0.0,292.0,68.0,162.0,197.0


In [48]:
df_photo = pd.read_csv('./newdb/photo.csv')
df_photo['image'] = df_photo["imagepath"].map(lambda x: x.split('/')[2].split('.')[0])

In [56]:

photo_ID_list = []
count = 0
for i in range(len(result_df)): #df_list[0] for Chia-wei Woo (吳家瑋)
    id = result_df.loc[i, "photoID"]
    aid = result_df.loc[i, "albumID"]
    c1, c2, c3, c4 = (int(i) for i in result_df.iloc[i, -4:])
    temp = df_photo[(df_photo['image'] == id) & (df_photo['aID'] == aid)]
    if not temp.empty:
        url = "https://digitalimages.hkust.edu.hk/gallery/"+temp['imagepath'].values[0].replace(' ', '%20')
        with urllib.request.urlopen(url) as url:
            s = url.read()
        arr = np.asarray(bytearray(s), dtype=np.uint8)
        img = cv2.imdecode(arr, -1) # 'Load it as it is'
        c1 = int(c1 * (img.shape[1]/640))
        c2 = int(c2 * (img.shape[1]/640))
        c3 = int(c3 * (img.shape[1]/640))
        c4 = int(c4 * (img.shape[1]/640))
        cv2.imwrite('raw/'  + id + "_" + str(count) + '.jpg', img)

        cv2.rectangle(img, (c1, c2), (c1+c3, c2+c4), (0, 0, 255), 2) 
        face = img[c2:c2 + c4, c1:c1 + c3]
        cv2.imwrite('unknown/rectangle/'  + id + "_cropped-" + str(count) + '.jpg', face)
        count += 1
        photo_ID_list.append([id +'.jpg'])

In [ ]:
url = "https://digitalimages.hkust.edu.hk/gallery/"+temp['imagepath'].values[0].replace(' ', '%20')
with urllib.request.urlopen(url) as url:
    s = url.read()

In [57]:
import os
current_directory = os.getcwd()
INPUT_DIR = os.path.join(current_directory, "raw")
OUTPUT_DIR = os.path.join(current_directory, "unknown/face-crop-plus")

# Set all to False if running on CPU (unless you can wait for a bit)
TEST_QUALITY_ENHANCEMENT = False
TEST_ATTR_GROUPING = False
TEST_MASK_GROUPING = False

if __name__ == "__main__":
    # Initialize as None
    enh_threshold = None
    attr_groups = None
    mask_groups = None

    if TEST_QUALITY_ENHANCEMENT:
        enh_threshold = 0.001

    if TEST_ATTR_GROUPING:
        attr_groups = {"hat": [18], "no_accessories": [-6, -9, -15, -18]}
    
    if TEST_MASK_GROUPING:
        mask_groups = {"nose": [10], "eyes_and_eyebrows": [2, 3, 4, 5]}

    # Initialize cropper
    cropper = Cropper(
        output_size=(256, 256),
        output_format="jpg",
        face_factor=0.7,
        strategy="all",
        device = "cuda:0" if is_available() else "cpu",
        enh_threshold=enh_threshold,
        attr_groups=attr_groups,
        mask_groups=mask_groups,
    )


    # Process images in the input dir and save face images to output dir
    cropper.process_dir(input_dir=INPUT_DIR, output_dir=OUTPUT_DIR)

Processing: 100%|██████████| 6/6 [01:58<00:00, 19.74s/it]


In [9]:
for i in range(1, 4):
    for unknown_img in os.path.join(current_directory, "unknown/face-crop-plus"):
        known_img = "known/" + "Chia-wei Woo (吳家瑋)" + "/" + "WOO-Chia-Wei_" + i + ".jpg"
        unknown_img = "ng5.jpg"
        model_name = 'Facenet'
        result = DeepFace.verify(known_img, unknown_img, model_name)
        print(result["verified"])

24-05-10 14:13:14 - facenet_weights.h5 will be downloaded...


ConnectionError: HTTPSConnectionPool(host='github.com', port=443): Max retries exceeded with url: /serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020FE2DD9BD0>: Failed to resolve 'github.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
def read_img(path):
    img = cv2.imread(path)
    (h, w) = img.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(img, (width, height))




acc_list = []

known_encodings = []
known_names = []
known_dir = 'known' # +  '/' + tagged_person
for file in os.listdir(known_dir):
  img = read_img(known_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)[0]
  known_encodings.append(img_enc)
  known_names.append(file.split('.')[0])

unknown_dir = 'unknown/face-crop-plus'
for file in os.listdir(unknown_dir):
  # print("Processing", file)
  img = read_img(unknown_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)
  if(img_enc == []):
    acc_list.append(0)
    continue

  results = face_recognition.compare_faces(known_encodings, img_enc[0])
  accuracy = np.sum(np.array(results)) / len(results)
  if(accuracy > 0.75): # change
    acc_list.append(1)
  else:
    acc_list.append(0)



([0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1],
 0.4523809523809524)

In [ ]:
tagged_person = tagged_person_list[0][:-6] # @@@replace with@@@ for tagged_person in tagged_person_list: 

record_list = []
for index in range(len(photo_ID_list)):
    file_name = photo_ID_list[index][0]
    cropped_file_name = file_name[:-4] +  '_cropped-' + str(index+1) + '.jpg'
    correct = acc_list[index]
    if correct:
        detected_person = tagged_person
    else:
        detected_person = 0
    record_list.append([file_name, tagged_person, cropped_file_name, detected_person, correct])

result = pd.DataFrame(record_list)


    

23

In [ ]:

result.to_csv("name_tag.csv")
